# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [36]:
import os
import io
import pandas as pd
import gzip
import mygene
import re
import tempfile
import shutil

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from typing import Callable, Optional, List, Tuple, Union, Any
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sparse_lmm import VariableSelection
from statsmodels.stats.multitest import multipletests
from sklearn.metrics import accuracy_score, mean_squared_error

print("utils.py has been loaded")
def geo_get_relevant_filepaths(cohort_dir):
    """Find the file paths of a SOFT file and a matrix file from the given data directory of a cohort.
    If there are multiple SOFT files or matrix files, simply choose the first one. Used for the GEO dataset.
    """
    files = os.listdir(cohort_dir)
    soft_files = [f for f in files if 'soft' in f.lower()]
    matrix_files = [f for f in files if 'matrix' in f.lower()]
    assert len(soft_files) > 0 and len(matrix_files) > 0
    soft_file_path = os.path.join(cohort_dir, soft_files[0])
    matrix_file_path = os.path.join(cohort_dir, matrix_files[0])

    return soft_file_path, matrix_file_path


def xena_get_relevant_filepaths(cohort_dir):
    """Find the file paths of a clinical file and a genetic file from the given data directory of a cohort.
    If there are multiple clinical or genetic data files, simply choose the first one. Used for the TCGA Xena dataset.
    """
    files = os.listdir(cohort_dir)
    clinical_files = [f for f in files if 'clinicalmatrix' in f.lower()]
    genetic_files = [f for f in files if 'pancan' in f.lower()]
    clinical_file_path = os.path.join(cohort_dir, clinical_files[0])
    genetic_file_path = os.path.join(cohort_dir, genetic_files[0])
    return clinical_file_path, genetic_file_path

def line_generator(source, source_type):
    """Generator that yields lines from a file or a string.

    Parameters:
    - source: File path or string content.
    - source_type: 'file' or 'string'.
    """
    if source_type == 'file':
        with gzip.open(source, 'rt') as f:
            for line in f:
                yield line.strip()
    elif source_type == 'string':
        for line in source.split('\n'):
            yield line.strip()
    else:
        raise ValueError("source_type must be 'file' or 'string'")


def filter_content_by_prefix(
    source: str,
    prefixes_a: List[str],
    prefixes_b: Optional[List[str]] = None,
    unselect: bool = False,
    source_type: str = 'file',
    return_df_a: bool = True,
    return_df_b: bool = True
) -> Tuple[Union[str, pd.DataFrame], Optional[Union[str, pd.DataFrame]]]:
    """
    Filters rows from a file or a list of strings based on specified prefixes.

    Parameters:
    - source (str): File path or string content to filter.
    - prefixes_a (List[str]): Primary list of prefixes to filter by.
    - prefixes_b (Optional[List[str]]): Optional secondary list of prefixes to filter by.
    - unselect (bool): If True, selects rows that do not start with the specified prefixes.
    - source_type (str): 'file' if source is a file path, 'string' if source is a string of text.
    - return_df_a (bool): If True, returns filtered content for prefixes_a as a pandas DataFrame.
    - return_df_b (bool): If True, and if prefixes_b is provided, returns filtered content for prefixes_b as a pandas DataFrame.

    Returns:
    - Tuple: A tuple where the first element is the filtered content for prefixes_a, and the second element is the filtered content for prefixes_b.
    """
    filtered_lines_a = []
    filtered_lines_b = []
    prefix_set_a = set(prefixes_a)
    if prefixes_b is not None:
        prefix_set_b = set(prefixes_b)

    # Use generator to get lines
    for line in line_generator(source, source_type):
        matched_a = any(line.startswith(prefix) for prefix in prefix_set_a)
        if matched_a != unselect:
            filtered_lines_a.append(line)
        if prefixes_b is not None:
            matched_b = any(line.startswith(prefix) for prefix in prefix_set_b)
            if matched_b != unselect:
                filtered_lines_b.append(line)

    filtered_content_a = '\n'.join(filtered_lines_a)
    if return_df_a:
        filtered_content_a = pd.read_csv(io.StringIO(filtered_content_a), delimiter='\t', low_memory=False, on_bad_lines='skip')
    filtered_content_b = None
    if filtered_lines_b:
        filtered_content_b = '\n'.join(filtered_lines_b)
        if return_df_b:
            filtered_content_b = pd.read_csv(io.StringIO(filtered_content_b), delimiter='\t', low_memory=False, on_bad_lines='skip')

    return filtered_content_a, filtered_content_b


def get_background_and_clinical_data(file_path,
                                     prefixes_a=['!Series_title', '!Series_summary', '!Series_overall_design'],
                                     prefixes_b=['!Sample_geo_accession', '!Sample_characteristics_ch1']):
    """Extract from a matrix file the background information about the dataset, and sample characteristics data"""
    background_info, clinical_data = filter_content_by_prefix(file_path, prefixes_a, prefixes_b, unselect=False,
                                                              source_type='file',
                                                              return_df_a=False, return_df_b=True)
    return background_info, clinical_data


def get_gene_annotation(file_path, prefixes=['^', '!', '#']):
    """Extract from a SOFT file the gene annotation data"""
    gene_metadata = filter_content_by_prefix(file_path, prefixes_a=prefixes, unselect=True, source_type='file',
                                             return_df_a=True)
    return gene_metadata[0]


def get_gene_mapping(annotation, prob_col, gene_col):
    """Process the gene annotation to get the mapping between gene names and gene probes.
    """
    mapping_data = annotation.loc[:, [prob_col, gene_col]]
    mapping_data = mapping_data.dropna()
    mapping_data = mapping_data.rename(columns={gene_col: 'Gene'}).astype({'ID': 'str'})

    return mapping_data


def get_genetic_data(file_path):
    """Read the gene expression data into a dataframe, and adjust its format"""
    genetic_data = pd.read_csv(file_path, compression='gzip', skiprows=52, comment='!', delimiter='\t')
    genetic_data = genetic_data.dropna()
    genetic_data = genetic_data.rename(columns={'ID_REF': 'ID'}).astype({'ID': 'str'})
    genetic_data.set_index('ID', inplace=True)

    return genetic_data


def apply_gene_mapping(expression_df, mapping_df):
    """
    Converts measured data about gene probes into gene expression data.
    Handles the potential many-to-many relationship between probes and genes.

    Parameters:
    expression_df (DataFrame): A DataFrame with gene expression data, indexed by 'ID'.
    mapping_df (DataFrame): A DataFrame mapping 'ID' to 'Gene', with 'ID' as a column.

    Returns:
    DataFrame: A DataFrame with mean gene expression values, indexed by 'Gene'.
    """

    # Define a regex pattern for splitting gene names
    split_pattern = r';|\|+|/{2,}|,|\[|\]|\(|\)'

    # Split the 'Gene' column in 'mapping_df' using the regex pattern
    mapping_df['Gene'] = mapping_df['Gene'].str.split(split_pattern)
    mapping_df = mapping_df.explode('Gene')

    # Set 'ID' as the index of 'mapping_df' for merging
    mapping_df.set_index('ID', inplace=True)

    # Merge 'mapping_df' with 'expression_df' by their indices
    merged_df = mapping_df.join(expression_df)

    # Group by 'Gene' and calculate the mean expression values
    gene_expression_df = merged_df.groupby('Gene').mean().dropna()

    return gene_expression_df


def normalize_gene_symbols(gene_symbols, batch_size=1000):
    """Normalize human gene symbols in batches using the 'mygenes' library"""
    mg = mygene.MyGeneInfo()
    normalized_genes = {}

    # Process in batches
    for i in range(0, len(gene_symbols), batch_size):
        batch = gene_symbols[i:i + batch_size]
        results = mg.querymany(batch, scopes='symbol', fields='symbol', species='human', verbose=False)

        # Update the normalized_genes dictionary with results from this batch
        for gene in results:
            normalized_genes[gene['query']] = gene.get('symbol', None)

    # Return the normalized symbols in the same order as the input
    return [normalized_genes.get(symbol) for symbol in gene_symbols]


def normalize_gene_symbols_in_index(gene_df):
    """Normalize the human gene symbols at the index of a dataframe, and replace the index with its normalized version.
    Remove the rows where the index failed to be normalized."""
    normalized_gene_list = normalize_gene_symbols(gene_df.index.tolist())
    assert len(normalized_gene_list) == len(gene_df.index)
    gene_df.index = normalized_gene_list
    gene_df = gene_df[gene_df.index.notnull()]
    return gene_df


def get_feature_data(clinical_df, row_id, feature, convert_fn):
    """select the row corresponding to a feature in the sample characteristics dataframe, and convert the feature into
    a binary or continuous variable"""
    clinical_df = clinical_df.iloc[row_id:row_id + 1].drop(columns=['!Sample_geo_accession'], errors='ignore')
    clinical_df.index = [feature]
    clinical_df = clinical_df.applymap(convert_fn)

    return clinical_df


def plot_numeric_distribution(df, column):
    """Plot the distribution of a numeric variable stored in the given column of a dataframe"""
    plt.figure(figsize=(10, 6))
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Distribution of {column.capitalize()}')
    plt.xlabel('')
    plt.ylabel('Frequency')
    plt.show()


def plot_categorical_distribution(df, column):
    """Plot the distribution of a categorical variable stored in the given column of a dataframe"""
    plt.figure(figsize=(10, 6))
    sns.countplot(y=column, data=df, order=df[column].value_counts().index)
    plt.title(f'Distribution of {column.capitalize()}')
    plt.xlabel('Frequency')
    plt.ylabel('')
    plt.show()


def analyze_distributions(df, numerical_columns, categorical_columns):
    """Plot the distribution of the numerical and categorical variables stored in given columns of a dataframe"""
    for col in numerical_columns:
        plot_numeric_distribution(df, col)

    for col in categorical_columns:
        plot_categorical_distribution(df, col)


def normalize_data(X_train, X_test=None):
    """Compute the mean and standard deviation statistics of the training data, use them to normalize the training data,
    and optionally the test data"""
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)

    # Handling columns with std = 0
    std_no_zero = np.where(std == 0, 1, std)

    # Normalize X_train
    X_train_normalized = (X_train - mean) / std_no_zero
    # Set normalized values to 0 where std was 0
    X_train_normalized[:, std == 0] = 0

    if X_test is not None:
        X_test_normalized = (X_test - mean) / std_no_zero
        X_test_normalized[:, std == 0] = 0
    else:
        X_test_normalized = None

    return X_train_normalized, X_test_normalized


class ResidualizationRegressor:
    def __init__(self, regression_model_constructor, params=None):
        if params is None:
            params = {}
        self.regression_model = regression_model_constructor(**params)
        self.beta_Z = None  # Coefficients for regression of Y on Z
        self.beta_X = None  # Coefficients for regression of residual on X
        self.neg_log_p_values = None  # Negative logarithm of p-values
        self.p_values = None  # Actual p-values

    def _reshape_data(self, data):
        """
        Reshape the data to ensure it's in the correct format (2D array).

        :param data: The input data (can be 1D or 2D array).
        :return: Reshaped 2D array.
        """
        if data.ndim == 1:
            return data.reshape(-1, 1)
        return data

    def _reshape_output(self, data):
        """
        Reshape the output data to ensure it's in the correct format (1D array).

        :param data: The output data (can be 1D or 2D array).
        :return: Reshaped 1D array.
        """
        if data.ndim == 2 and data.shape[1] == 1:
            return data.ravel()
        return data

    def fit(self, X, Y, Z):
        X = self._reshape_data(X)
        Y = self._reshape_data(Y)
        Z = self._reshape_data(Z)

        # Step 1: Linear regression of Y on Z
        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        self.beta_Z = np.linalg.pinv(Z_ones.T @ Z_ones) @ Z_ones.T @ Y
        Y_hat = Z_ones @ self.beta_Z
        e_Y = Y - Y_hat  # Residual of Y

        # Step 2: Regress the residual on X using the included regression model
        self.regression_model.fit(X, e_Y)

        # Obtain coefficients from the regression model
        if hasattr(self.regression_model, 'coef_'):
            self.beta_X = self.regression_model.coef_
        elif hasattr(self.regression_model, 'getBeta'):
            beta_output = self.regression_model.getBeta()
            self.beta_X = self._reshape_output(beta_output)

        # Obtain negative logarithm of p-values, if available
        if hasattr(self.regression_model, 'getNegLogP'):
            neg_log_p_output = self.regression_model.getNegLogP()
            if neg_log_p_output is not None:
                self.neg_log_p_values = self._reshape_output(neg_log_p_output)
                self.p_values = np.exp(-self.neg_log_p_values)
                # Concatenate the p-values of Z and X. The p-values of Z were not computed, mark with NaN.
                p_values_Z = np.full(Z.shape[1], np.nan)
                self.p_values = np.concatenate((p_values_Z, self.p_values))

    def predict(self, X, Z):
        X = self._reshape_data(X)
        Z = self._reshape_data(Z)

        Z_ones = np.column_stack((np.ones(Z.shape[0]), Z))
        ZX = np.column_stack((Z, X))
        combined_beta = np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))
        return ZX @ combined_beta + self.beta_Z[0]

    def get_coefficients(self):
        return np.concatenate((self.beta_Z[1:].ravel(), self.beta_X.ravel()))

    def get_p_values(self):
        return self.p_values


def cross_validation(X, Y, Z, model_constructor, model_params, k=5, target_type='binary'):
    assert target_type in ['binary', 'continuous'], "The target type must be chosen from 'binary' or 'continuous'"
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    fold_size = len(X) // k
    performances = []

    for i in range(k):
        # Split data into train and test based on the current fold
        test_indices = indices[i * fold_size: (i + 1) * fold_size]
        train_indices = np.setdiff1d(indices, test_indices)

        X_train, X_test = X[train_indices], X[test_indices]
        Y_train, Y_test = Y[train_indices], Y[test_indices]
        Z_train, Z_test = Z[train_indices], Z[test_indices]

        normalized_X_train, normalized_X_test = normalize_data(X_train, X_test)
        normalized_Z_train, normalized_Z_test = normalize_data(Z_train, Z_test)

        # model = model_constructor(**model_params)
        model = ResidualizationRegressor(model_constructor, model_params)
        model.fit(normalized_X_train, Y_train, normalized_Z_train)
        predictions = model.predict(normalized_X_test, normalized_Z_test)

        if target_type == 'binary':
            predictions = (predictions > 0.5).astype(int)
            Y_test = (Y_test > 0.5).astype(int)
            performance = accuracy_score(Y_test, predictions)
        elif target_type == 'continuous':
            performance = mean_squared_error(Y_test, predictions)

        performances.append(performance)

    cv_mean = np.mean(performances)
    cv_std = np.std(performances)

    if target_type == 'binary':
        print(f'The cross-validation accuracy is {(cv_mean * 100):.2f}% ± {(cv_std * 100):.2f}%')
    else:
        print(f'The cross-validation MSE is {(cv_mean * 100):.2f} ± {(cv_std * 100):.2f}')

    return cv_mean, cv_std


def get_feature_related_genes(file_path, feature, normalize):
    """Read a csv file into a dataframe about gene-trait association, and get the gene symbols related to a given
    trait"""
    related_gene_df = pd.read_csv(file_path)
    related_gene_df = related_gene_df.loc[:, ['Disease Name', 'Corresponding_Gene_Symbol']].set_index('Disease Name')
    feature_related_genes = related_gene_df.loc[feature].tolist()[0].strip().split(',')
    feature_related_genes = [gn.strip() for gn in feature_related_genes]
    if normalize:
        feature_related_genes = normalize_gene_symbols(feature_related_genes)

    return feature_related_genes


def get_gene_regressors(trait, trait_df, condition_df, related_genes):
    """Find the appropriate genes for two-step regression. Compare the indices of two dataframes to find the genes
    in common, and select those that are known to be related to a trait"""
    gene_regressors = None
    genes_in_trait_data = set(trait_df.columns) - {'Age', 'Gender', trait}
    genes_in_condition_data = set(condition_df.columns) - {'Age', 'Gender', trait}

    common_genes_across_data = genes_in_trait_data.intersection(genes_in_condition_data)
    if len(common_genes_across_data) == 0:
        #print("The trait and condition datasets have no genes in common. Please try other datasets")
        return None
    else:
        ##print(
            ## f"The trait and condition datasets have {len(common_genes_across_data)} genes in common, such as {list(common_genes_across_data)[:10]}.")
        common_genes = [g for g in related_genes if g in common_genes_across_data]
        if len(common_genes) > 0:
            gene_regressors = list(common_genes)[:10]
            #print(
                #f"Found {len(common_genes)} candidate genes that can be used in two-step regression analysis, such as {gene_regressors[:10]}.")
        else:
            #print(
                #f"The condition and trait datasets have common genes, but among them we didn't find indicator genes for the condition")
            return None

    return gene_regressors


def normalize_trait(trait):
    trait = '-'.join(trait.split())
    normalized_trait = ''.join(trait.split("'"))
    return normalized_trait

def interpret_result(model: Any, feature_names: List[str], trait: str, condition: str,
                     threshold: float = 0.05, save_output: bool = True,
                     output_dir: str = './output', model_id: int = 1) -> None:
    """This function interprets and reports the result of a trained linear regression model, where the regressor
    consists of one variable about condition and multiple variables about genetic factors.
    The function extracts coefficients and p-values from the model, and identifies the significant genes based on
    p-values or non-zero coefficients, depending on the availability of p-values.

    Parameters:
    model (Any): The trained regression Model.
    feature_names (List[str]): A list of feature names corresponding to the model's coefficients.
    trait (str): The target trait of interest.
    condition (str): The specific condition to examine within the model.
    threshold (float): Significance level for p-value correction. Defaults to 0.05.
    save_output (bool): Flag to determine whether to save the output to a file. Defaults to True.
    output_dir (str): Directory path where output files are saved. Defaults to './output'.
    model_id (int): The index of the model, 1 or 2.

    Returns:
    None: This function does not return anything but prints and optionally saves the output.
    """
    coefficients = model.get_coefficients().reshape(-1).tolist()
    p_values = model.get_p_values()
    if p_values is None:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients
        })
    else:
        regression_df = pd.DataFrame({
            'Variable': feature_names,
            'Coefficient': coefficients,
            'p_value': p_values.reshape(-1).tolist()
        })

    condition_effect = regression_df[regression_df['Variable'] == condition].iloc[0]

    print(f"Effect of the condition on the target variable:")
    print(f"Variable: {condition}")
    print(f"Coefficient: {condition_effect['Coefficient']:.4f}")
    gene_regression_df = regression_df[regression_df['Variable'] != condition]
    if p_values is None:
        gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()
        significant_genes = gene_regression_df[gene_regression_df['Coefficient'] != 0]
        significant_genes_sorted = significant_genes.sort_values(by='Absolute Coefficient', ascending=False)
        print(
            f"Found {len(significant_genes_sorted)} genes with non-zero coefficients associated with the trait '{trait}' "
            f"conditional on the factor '{condition}'. These genes are identified as significant based on the regression model.")
    else:
        # Apply the Benjamini-Hochberg correction, to get the corrected p-values
        corrected_p_values = multipletests(gene_regression_df['p_value'], alpha=threshold, method='fdr_bh')[1]
        gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
        significant_genes = gene_regression_df.loc[gene_regression_df['corrected_p_value'] < threshold]
        significant_genes_sorted = significant_genes.sort_values('corrected_p_value')
        print(
            f"Found {len(significant_genes_sorted)} significant genes associated with the trait '{trait}' conditional on "
            f"the factor '{condition}', with corrected p-value < {threshold}:")

    print(significant_genes_sorted.to_string(index=False))

    nm_condition = normalize_trait(condition)
    # Optionally, save this to a CSV file
    if save_output:
        significant_genes_sorted.to_csv(
            os.path.join(output_dir, f'significant_genes_condition_{nm_condition}_{model_id}.csv'), index=False)


def judge_binary_variable_biased(dataframe, col_name, min_proportion=0.1, min_num=5):
    """
    Check if the distribution of a binary variable in the dataset is too biased to be usable for analysis
    :param dataframe:
    :param col_name:
    :param min_proportion:
    :param min_num:
    :return:
    """
    label_counter = dataframe[col_name].value_counts()
    total_samples = len(dataframe)
    rare_label_num = label_counter.min()
    rare_label = label_counter.idxmin()
    rare_label_proportion = rare_label_num / total_samples

    print(
        f"For the feature \'{col_name}\', the least common label is '{rare_label}' with {rare_label_num} occurrences. This represents {rare_label_proportion:.2%} of the dataset.")

    biased = (len(label_counter) < 2) or ((rare_label_proportion < min_proportion) and (rare_label_num < min_num))
    return bool(biased)


def judge_continuous_variable_biased(dataframe, col_name):
    """Check if the distribution of a continuous variable in the dataset is too biased to be usable for analysis.
    As a starting point, we consider it biased if all values are the same. For the next step, maybe ask GPT to judge
    based on quartile statistics combined with its common sense knowledge about this feature.
    """
    quartiles = dataframe[col_name].quantile([0.25, 0.5, 0.75])
    min_value = dataframe[col_name].min()
    max_value = dataframe[col_name].max()

    # Printing quartile information
    print(f"Quartiles for '{col_name}':")
    print(f"  25%: {quartiles[0.25]}")
    print(f"  50% (Median): {quartiles[0.5]}")
    print(f"  75%: {quartiles[0.75]}")
    print(f"Min: {min_value}")
    print(f"Max: {max_value}")

    biased = min_value == max_value

    return bool(biased)


def check_rows_and_columns(dataframe, display=False):
    """
    Get the lists of row names and column names of a dataset, and optionally observe them.
    :param dataframe:
    :param display:
    :return:
    """
    dataframe_rows = dataframe.index.tolist()
    if display:
        print(f"The dataset has {len(dataframe_rows)} rows, such as {dataframe_rows[:20]}")
    dataframe_cols = dataframe.columns.tolist()
    if display:
        print(f"\nThe dataset has {len(dataframe_cols)} columns, such as {dataframe_cols[:20]}")
    return dataframe_rows, dataframe_cols


def xena_convert_trait(row_index: str):
    """
    Convert the trait information from Sample IDs to labels depending on the last two digits.
    Tumor types range from 01 - 09, normal types from 10 - 19.
    :param row_index: the index value of a row
    :return: the converted value
    """
    last_two_digits = int(row_index[-2:])

    if 1 <= last_two_digits <= 9:
        return 1
    elif 10 <= last_two_digits <= 19:
        return 0
    else:
        return -1


def xena_convert_gender(cell: str):
    """Convert the cell content about gender to a binary value
    """
    if isinstance(cell, str):
        cell = cell.lower()

    if cell == "female":
        return 0
    elif cell == "male":
        return 1
    else:
        return None


def xena_convert_age(cell: str):
    """Convert the cell content about age to a numerical value using regular expression
    """
    match = re.search(r'\d+', str(cell))
    if match:
        return int(match.group())
    else:
        return None


def detect_batch_effect(X):
    """
    Detect potential batch effects in a dataset using eigenvalues of XX^T.

    Args:
    X (numpy.ndarray): A feature matrix with shape (n_samples, n_features).

    Returns:
    bool: True if a potential batch effect is detected, False otherwise.
    """
    n_samples = X.shape[0]

    # Computing XX^T
    XXt = np.dot(X, X.T)

    # Compute the eigenvalues of XX^T
    eigen_values = np.linalg.eigvalsh(XXt)  # Using eigvalsh since XX^T is symmetric
    eigen_values = sorted(eigen_values, reverse=True)[:10]
    eigen_values = np.array(eigen_values)
    normalized_ev = eigen_values / eigen_values[0]

    # Check for large gaps in the eigenvalues
    for i in range(len(normalized_ev) - 1):
        gap = normalized_ev[i] - normalized_ev[i + 1]
        if gap > 1 / n_samples:  # You may need to adjust this threshold
            return True

    return False


def get_unique_values_by_row(dataframe, max_len=30):
    """
    Organize the unique values in each row of the given dataframe, to get a dictionary
    :param dataframe:
    :param max_len:
    :return:
    """
    if '!Sample_geo_accession' in dataframe.columns:
        dataframe = dataframe.drop(columns=['!Sample_geo_accession'])
    unique_values_dict = {}
    for index, row in dataframe.iterrows():
        unique_values = list(row.unique())[:max_len]
        unique_values_dict[index] = unique_values
    return unique_values_dict


def xena_select_clinical_features(clinical_df, trait, age_col=None, gender_col=None):
    feature_list = []
    trait_data = clinical_df.index.to_series().apply(xena_convert_trait).rename(trait)
    feature_list.append(trait_data)
    if age_col:
        age_data = clinical_df[age_col].apply(xena_convert_age).rename("Age")
        feature_list.append(age_data)
    if gender_col:
        gender_data = clinical_df[gender_col].apply(xena_convert_gender).rename("Gender")
        feature_list.append(gender_data)
    selected_clinical_df = pd.concat(feature_list, axis=1)
    return selected_clinical_df


def geo_select_clinical_features(clinical_df: pd.DataFrame, trait: str, trait_row: int,
                                 convert_trait: Callable,
                                 age_row: Optional[int] = None,
                                 convert_age: Optional[Callable] = None,
                                 gender_row: Optional[int] = None,
                                 convert_gender: Optional[Callable] = None) -> pd.DataFrame:
    """
    Extracts and processes specific clinical features from a DataFrame representing
    sample characteristics in the GEO database series.

    Parameters:
    - clinical_df (pd.DataFrame): DataFrame containing clinical data.
    - trait (str): The trait of interest.
    - trait_row (int): Row identifier for the trait in the DataFrame.
    - convert_trait (Callable): Function to convert trait data into a desired format.
    - age_row (int, optional): Row identifier for age data. Default is None.
    - convert_age (Callable, optional): Function to convert age data. Default is None.
    - gender_row (int, optional): Row identifier for gender data. Default is None.
    - convert_gender (Callable, optional): Function to convert gender data. Default is None.

    Returns:
    pd.DataFrame: A DataFrame containing the selected and processed clinical features.
    """
    feature_list = []

    trait_data = get_feature_data(clinical_df, trait_row, trait, convert_trait)
    feature_list.append(trait_data)
    if age_row is not None:
        age_data = get_feature_data(clinical_df, age_row, 'Age', convert_age)
        feature_list.append(age_data)
    if gender_row is not None:
        gender_data = get_feature_data(clinical_df, gender_row, 'Gender', convert_gender)
        feature_list.append(gender_data)

    selected_clinical_df = pd.concat(feature_list, axis=0)
    return selected_clinical_df


def geo_merge_clinical_genetic_data(clinical_df, genetic_df):
    """
    Merge the clinical features and gene expression features from two dataframes into one dataframe
    """
    if 'ID' in genetic_df.columns:
        genetic_df = genetic_df.rename(columns={'ID': 'Gene'})
    if 'Gene' in genetic_df.columns:
        genetic_df = genetic_df.set_index('Gene')
    merged_data = pd.concat([clinical_df, genetic_df], axis=0).T.dropna()
    return merged_data


def judge_and_remove_biased_features(df, trait, trait_type):
    assert trait_type in ["binary", "continuous"], f"The trait must be either a binary or a continuous variable!"
    if trait_type == "binary":
        trait_biased = judge_binary_variable_biased(df, trait)
    else:
        trait_biased = judge_continuous_variable_biased(df, trait)
    if trait_biased:
        print(f"The distribution of the feature \'{trait}\' in this dataset is severely biased.\n")
    else:
        print(f"The distribution of the feature \'{trait}\' in this dataset is fine.\n")
    if "Age" in df.columns:
        age_biased = judge_continuous_variable_biased(df, 'Age')
        if age_biased:
            print(f"The distribution of the feature \'Age\' in this dataset is severely biased.\n")
            df = df.drop(columns='Age')
        else:
            print(f"The distribution of the feature \'Age\' in this dataset is fine.\n")
    if "Gender" in df.columns:
        gender_biased = judge_binary_variable_biased(df, 'Gender')
        if gender_biased:
            print(f"The distribution of the feature \'Gender\' in this dataset is severely biased.\n")
            df = df.drop(columns='Gender')
        else:
            print(f"The distribution of the feature \'Gender\' in this dataset is fine.\n")

    return trait_biased, df


def save_cohort_info(cohort: str, info_path: str, is_available: bool, is_biased: Optional[bool] = None,
                     df: Optional[pd.DataFrame] = None, note: str = '') -> None:
    """
    Add or update information about the usability and quality of a dataset for statistical analysis.

    Parameters:
    cohort (str): A unique identifier for the dataset.
    info_path (str): File path to the JSON file where records are stored.
    is_available (bool): Indicates whether both the genetic data and trait data are available in the dataset, and can be
     preprocessed into a dataframe.
    is_biased (bool, optional): Indicates whether the dataset is too biased to be usable.
        Required if `is_available` is True.
    df (pandas.DataFrame, optional): The preprocessed dataset. Required if `is_available` is True.
    note (str, optional): Additional notes about the dataset.

    Returns:
    None: The function does not return a value but updates or creates a record in the specified JSON file.
    """
    if is_available:
        assert (df is not None) and (is_biased is not None), "'df' and 'is_biased' should be provided if this cohort " \
                                                             "is relevant."
    is_usable = is_available and (not is_biased)
    new_record = {"is_usable": is_usable,
                  "is_available": is_available,
                  "is_biased": is_biased if is_available else None,
                  "has_age": "Age" in df.columns if is_available else None,
                  "has_gender": "Gender" in df.columns if is_available else None,
                  "sample_size": len(df) if is_available else None,
                  "note": note}

    trait_directory = os.path.dirname(info_path)
    os.makedirs(trait_directory, exist_ok=True)
    if not os.path.exists(info_path):
        with open(info_path, 'w') as file:
            json.dump({}, file)
        print(f"A new JSON file was created at: {info_path}")

    with open(info_path, "r") as file:
        records = json.load(file)
    records[cohort] = new_record

    temp_path = info_path + ".tmp"
    try:
        with open(temp_path, 'w') as file:
            json.dump(records, file)
        os.replace(temp_path, info_path)

    except Exception as e:
        print(f"An error occurred: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        raise


def read_json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a pandas DataFrame.

    Args:
    json_file (str): The path to the JSON file containing the data.

    Returns:
    DataFrame: A pandas DataFrame with the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'cohort_id'})


def filter_and_rank_cohorts(json_file: str, condition: Union[str, None] = None) -> Tuple[
    Union[str, None], pd.DataFrame]:
    """
    Reads a JSON file, filters cohorts based on usability and an optional condition, then ranks them by sample size.

    Args:
    json_file (str): The path to the JSON file containing the data.
    condition (str, optional): An additional condition for filtering. If None, only 'is_usable' is considered.

    Returns:
    Tuple: A tuple containing the best cohort ID (str or None if no suitable cohort is found) and
           the filtered and ranked DataFrame.
    """
    # Read the JSON file into a DataFrame
    df = read_json_to_dataframe(json_file)

    if condition:
        filtered_df = df[(df['is_usable'] == True) & (df[condition] == True)]
    else:
        filtered_df = df[df['is_usable'] == True]

    ranked_df = filtered_df.sort_values(by='sample_size', ascending=False)
    best_cohort_id = ranked_df.iloc[0]['cohort_id'] if not ranked_df.empty else None

    return best_cohort_id, ranked_df


def read_json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a pandas DataFrame.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'cohort_id'})


def filter_and_rank_cohorts(json_file: str, condition: Union[str, None] = None) -> Tuple[
    Union[str, None], pd.DataFrame]:
    """
    Reads a JSON file, filters cohorts based on usability and an optional condition, then ranks them by sample size.

    Args:
    json_file (str): The path to the JSON file containing the data.
    condition (str, optional): An additional condition for filtering. If None, only 'is_usable' is considered.

    Returns:
    Tuple: A tuple containing the best cohort ID (str or None if no suitable cohort is found) and
           the filtered and ranked DataFrame.
    """
    df = read_json_to_dataframe(json_file)
    if condition:
        if condition.lower() in ['age', 'gender']:
            condition = 'has_' + condition.lower()
        assert condition in ['has_age', 'has_gender']
        filtered_df = df[(df['is_usable'] == True) & (df[condition] == True)]
    else:
        filtered_df = df[df['is_usable'] == True]
    ranked_df = filtered_df.sort_values(by='sample_size', ascending=False)
    best_cohort_id = ranked_df.iloc[0]['cohort_id'] if not ranked_df.empty else None

    return best_cohort_id, ranked_df


def preview_df(df, n=5):
    return df.head(n).to_dict(orient='list')


utils.py has been loaded


In [37]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Jake"
# Set the data and output directories
DATA_ROOT = '/Users/jacob/Documents'
OUTPUT_ROOT = '/Users/jacob/Documents/Output'
TRAIT = 'Telomere Length'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = None
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

cohort_dir = os.path.join(DATA_ROOT, dataset, trait_subdir)
clinical_data_file, genetic_data_file = xena_get_relevant_filepaths(cohort_dir)

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

In [ ]:
candidate_age_cols = ['Age_at_Initial_Pathologic_Diagnosis_nature2012', 'age_at_initial_pathologic_diagnosis',
                      'days_to_birth', 'year_of_initial_pathologic_diagnosis']
candidate_gender_cols = ['Gender_nature2012', 'gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [38]:
dataset = 'GEO'
trait_subdir = "Telomere-Length"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE16058', 'GSE25864', 'GSE30460', 'GSE52237', 'GSE80435']

Repeat the below steps for all the accession numbers

In [407]:
cohort = accession_num = "GSE52237"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/jacob/Documents\\GEO\\Telomere-Length\\GSE52237\\GSE52237_family.soft.gz',
 '/Users/jacob/Documents\\GEO\\Telomere-Length\\GSE52237\\GSE52237_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [413]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Smoking accelerated aging of the small airway epithelium"
!Series_summary	"Aging involves multiple biologically complex processes characterized by a decline in cellular homeostasis over time leading to a loss and impairment of physiological integrity and function. Specific cellular hallmarks of aging include abnormal gene expression patterns, shortened telomeres and associated biological dysfunction. Like all organs, the lung demonstrates both physiological and structural changes with age that result in a progressive decrease in lung function in healthy individuals. Cigarette smoking accelerates lung function decline over time, suggesting smoking accelerates aging of the lung. Based on this data, we hypothesized that cigarette smoking accelerates the aging of the small airway epithelium, the cells that take the initial brunt of inhaled toxins from the cigarette smoke and one of the primary sites of pathology associated with cigarette smoking. Using the sensitive molecula

In [414]:
clinical_data.head()

,!Sample_geo_accession,GSM1056118,GSM1056119,GSM1056120,GSM1056123,GSM1056124,GSM1056125,GSM1056126,GSM1056127,GSM1056128,...,GSM1074767,GSM1074768,GSM1074769,GSM1074773,GSM1074774,GSM1074775,GSM1074778,GSM1261679,GSM1261680,GSM1261681
0,!Sample_characteristics_ch1,smoking status: S,smoking status: S,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: nonsmoker,...,smoking status: smoker,smoking status: smoker,smoking status: smoker,smoking status: smoker,smoking status: smoker,smoking status: smoker,smoking status: smoker,smoking status: nonsmoker,smoking status: nonsmoker,smoking status: smoker
1,!Sample_characteristics_ch1,NaN,NaN,NaN,cilia length: 7.7322,cilia length: 8.07239,cilia length: 6.15053,cilia length: 7.19703,cilia length: 7.40795,cilia length: 6.85311,...,NaN,cilia length: 6.59999,NaN,NaN,cilia length: 6.45331,cilia length: 6.70242,NaN,NaN,NaN,NaN


In [415]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['smoking status: S',
  'smoking status: nonsmoker',
  'smoking status: smoker'],
 1: [nan,
  'cilia length: 7.7322',
  'cilia length: 8.07239',
  'cilia length: 6.15053',
  'cilia length: 7.19703',
  'cilia length: 7.40795',
  'cilia length: 6.85311',
  'cilia length: 7.07429',
  'cilia length: 7.27673',
  'cilia length: 7.78135',
  'cilia length: 6.78668',
  'cilia length: 7.01968',
  'cilia length: 5.85311',
  'cilia length: 5.94677',
  'cilia length: 6.45982',
  'cilia length: 7.16176',
  'cilia length: 6.02311',
  'cilia length: 7.84439',
  'cilia length: 7.32101',
  'cilia length: 7.25948',
  'cilia length: 7.08977',
  'cilia length: 7.75275',
  'cilia length: 8.57827',
  'cilia length: 6.64001',
  'cilia length: 6.47906',
  'cilia length: 7.28888',
  'cilia length: 6.29608',
  'cilia length: 6.34788',
  'cilia length: 6.59999',
  'cilia length: 6.45331']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [416]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Telomere Length\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Telomere Length\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [417]:
is_gene_availabe = True
trait_row = 1
age_row = None 
gender_row = None 

trait_type = 'binary'

In [412]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [418]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'tissue: Hippocampus':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present


def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format


def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

In [419]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

C:\Users\jacob\AppData\Local\Temp\ipykernel_4296\2151059819.py:218: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM1056118,GSM1056119,GSM1056120,GSM1056123,GSM1056124,GSM1056125,GSM1056126,GSM1056127,GSM1056128,GSM1056129,...,GSM1074767,GSM1074768,GSM1074769,GSM1074773,GSM1074774,GSM1074775,GSM1074778,GSM1261679,GSM1261680,GSM1261681
Telomere Length,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [420]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1056118,GSM1056119,GSM1056120,GSM1056123,GSM1056124,GSM1056125,GSM1056126,GSM1056127,GSM1056128,GSM1056129,...,GSM1074767,GSM1074768,GSM1074769,GSM1074773,GSM1074774,GSM1074775,GSM1074778,GSM1261679,GSM1261680,GSM1261681
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,8841.9400,8968.8800,8574.5600,8711.8400,8379.9800,11019.4000,7760.30000,8630.8000,7119.4700,8132.9000,...,10244.9000,10575.30000,8913.3200,8154.2600,8901.6800,9842.2100,9052.4300,7854.9700,8881.67000,8352.08000
1053_at,215.7370,200.3830,238.7460,203.4450,245.3960,223.9030,254.16900,278.0420,247.7080,257.6380,...,174.1300,254.67900,343.1380,289.0150,259.0530,245.4000,275.6400,280.4860,218.53700,220.29200
117_at,58.3822,92.3078,49.9451,42.3288,97.0849,104.1250,88.66160,106.2600,73.1918,91.2122,...,155.2330,118.14900,142.1310,60.3355,117.3950,107.0880,37.0456,94.3787,78.02420,76.06640
121_at,425.3440,662.7990,459.1800,599.5560,410.6710,406.0780,409.81100,406.1220,392.2310,513.1950,...,375.3980,488.97400,501.3250,375.8340,708.3220,471.5320,680.7340,332.0170,621.91400,453.33300
1255_g_at,27.0730,30.0499,15.8687,42.9129,31.8592,17.4396,5.01749,37.7257,12.8729,34.0415,...,34.2036,6.63732,49.9993,47.4051,71.5999,15.5355,22.2387,34.4539,4.61193,6.54806


In [421]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552258_at',
 '1552261_at',
 '1552263_at',
 '1552264_a_at',
 '1552266_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [422]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']\n"

If not required, jump directly to the gene normalization step

In [428]:
requires_gene_mapping = True  

In [429]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

In [430]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')

gene_annotation.columns

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [431]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = key_name1
    gene_symbol_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = key_name1
    gene_symbol_key = key_name2

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, 

In [453]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'GB_ACC'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [434]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [435]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [436]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 58 samples.


In [437]:
merged_data.head()

,Telomere Length,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
GSM1056118,0.0,94.5241,106.4960,75.022700,238.48310,202.884,28.43480,42.66830,113.591,27.5786,...,374.39350,294.107,377.930,358.063100,446.759500,24.33170,927.834000,135.1480,380.180567,657.2865
GSM1056119,0.0,82.8026,161.3340,60.308900,318.93300,258.592,57.90420,35.43720,192.560,41.5656,...,356.30725,414.182,272.134,294.400750,353.182167,5.77857,829.890333,266.5615,368.597067,542.4970
GSM1056120,0.0,132.2160,75.7088,60.387500,263.89050,276.841,27.21225,46.17570,102.714,10.2208,...,310.45215,387.446,354.448,189.359820,452.446833,57.90850,923.555933,200.9070,333.009233,587.5085
GSM1056123,0.0,131.9180,12.2004,48.681805,266.08085,333.603,24.60185,28.38430,153.434,11.6618,...,349.79050,268.653,358.153,238.653815,498.183833,3.35408,863.459333,162.6285,265.795733,565.2460
GSM1056124,0.0,109.5250,99.6689,53.349355,231.18100,195.385,33.59125,7.94258,133.097,39.1729,...,334.56100,384.300,424.666,287.497300,460.200000,3.93480,933.435000,133.2760,305.004733,679.6775


In [438]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Telomere Length', the least common label is '0.0' with 58 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Telomere Length' in this dataset is severely biased.



True

In [439]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [440]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [441]:
from utils import *

In [458]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

In [462]:
def read_json_to_dataframe(json_file: str) -> pd.DataFrame:
    """
    Reads a JSON file and converts it into a pandas DataFrame.

    Args:
    json_file (str): The path to the JSON file containing the data.

    Returns:
    DataFrame: A pandas DataFrame with the JSON data.
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index').reset_index().rename(columns={'index': 'cohort_id'})

In [463]:
def filter_and_rank_cohorts(json_file: str, condition: Union[str, None] = None) -> Tuple[
    Union[str, None], pd.DataFrame]:
    """
    Reads a JSON file, filters cohorts based on usability and an optional condition, then ranks them by sample size.

    Args:
    json_file (str): The path to the JSON file containing the data.
    condition (str, optional): An additional condition for filtering. If None, only 'is_usable' is considered.

    Returns:
    Tuple: A tuple containing the best cohort ID (str or None if no suitable cohort is found) and
           the filtered and ranked DataFrame.
    """
    # Read the JSON file into a DataFrame
    df = read_json_to_dataframe(json_file)

    if condition:
        filtered_df = df[(df['is_usable'] == True) & (df[condition] == True)]
    else:
        filtered_df = df[df['is_usable'] == True]

    ranked_df = filtered_df.sort_values(by='sample_size', ascending=False)
    best_cohort_id = ranked_df.iloc[0]['cohort_id'] if not ranked_df.empty else None

    return best_cohort_id, ranked_df

In [464]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note


In [468]:
import pandas as pd
import os

# Assuming OUTPUT_DIR and best_cohort are defined earlier in your code
if best_cohort is not None:
    merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort + '.csv'))
    print(merged_data.head())
else:
    print("No suitable cohort found, best_cohort is None.")


No suitable cohort found, best_cohort is None.


In [467]:
merged_data

,Telomere Length,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
GSM1056118,0.0,94.5241,106.4960,75.022700,238.48310,202.884,28.434800,42.66830,113.5910,27.57860,...,374.39350,294.107,377.930,358.063100,446.759500,24.33170,927.834000,135.1480,380.180567,657.2865
GSM1056119,0.0,82.8026,161.3340,60.308900,318.93300,258.592,57.904200,35.43720,192.5600,41.56560,...,356.30725,414.182,272.134,294.400750,353.182167,5.77857,829.890333,266.5615,368.597067,542.4970
GSM1056120,0.0,132.2160,75.7088,60.387500,263.89050,276.841,27.212250,46.17570,102.7140,10.22080,...,310.45215,387.446,354.448,189.359820,452.446833,57.90850,923.555933,200.9070,333.009233,587.5085
GSM1056123,0.0,131.9180,12.2004,48.681805,266.08085,333.603,24.601850,28.38430,153.4340,11.66180,...,349.79050,268.653,358.153,238.653815,498.183833,3.35408,863.459333,162.6285,265.795733,565.2460
GSM1056124,0.0,109.5250,99.6689,53.349355,231.18100,195.385,33.591250,7.94258,133.0970,39.17290,...,334.56100,384.300,424.666,287.497300,460.200000,3.93480,933.435000,133.2760,305.004733,679.6775
GSM1056125,0.0,135.5830,73.2525,81.673880,179.29655,321.674,47.219550,51.85580,157.3300,10.23950,...,391.61350,265.132,254.586,233.221150,387.971667,18.75830,876.814667,175.1335,324.390000,582.6240
GSM1056126,0.0,186.0020,104.6750,55.429905,220.48550,240.189,33.309600,62.32860,85.0239,67.81080,...,287.74495,372.654,390.823,359.099200,368.730667,3.79172,963.678500,183.6740,249.835943,659.1240
GSM1056127,0.0,172.8250,144.7910,95.381000,381.15845,237.151,40.471900,41.31560,85.7032,71.35160,...,381.95250,363.157,365.508,303.516700,454.170667,9.68961,905.032667,311.8060,376.162333,679.4760
GSM1056128,0.0,111.1560,12.8451,48.629131,213.68770,194.475,16.429010,25.27260,56.8685,46.77840,...,354.67275,341.374,328.352,332.362400,398.612833,5.61608,859.973000,189.0910,289.542467,643.6700
GSM1056129,0.0,166.9280,126.0510,70.369250,260.68750,255.662,7.488690,46.05540,49.2034,7.81612,...,324.18850,314.115,394.993,411.303450,439.738167,5.57225,874.722200,180.4730,307.956097,663.4895


Select the appropriate regression model depending on whether the dataset shows batch effect.

In [449]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

C:\Users\jacob\AppData\Local\Temp\ipykernel_4296\2151059819.py:631: RuntimeWarning: invalid value encountered in divide
  normalized_ev = eigen_values / eigen_values[0]


False

In [450]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [469]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

NameError: name 'Z' is not defined

In [404]:
Y = merged_data[TRAIT].values


In [405]:
cv_mean2, cv_std2 = cross_validation(Y, model_constructor2, model_params2, target_type=trait_type)

TypeError: cross_validation() missing 2 required positional arguments: 'model_constructor' and 'model_params'

In [406]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

NameError: name 'Z' is not defined

### 4. Discussion and report

In [329]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

ValueError: list.remove(x): x not in list

In [37]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.3762
Found 0 significant genes associated with the trait 'Epilepsy' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/home/techt/Desktop/TAIS_2024SP/utils.py:495: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
